In [1]:
!pip install pytorch-crf

In [2]:
# BiLSTM - CRF Implementation

import torch
import torch.nn as nn
import torch.optim as optim
from torchcrf import CRF
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size, padding_idx):
        super(BiLSTM_CRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.crf = CRF(tagset_size, batch_first=True)

    def forward(self, sentences):
        embeds = self.embedding(sentences)
        lstm_out, _ = self.lstm(embeds)
        lstm_out = self.hidden2tag(lstm_out)
        return lstm_out

    def neg_log_likelihood(self, sentences, tags):
        lstm_out = self.forward(sentences)
        return -self.crf(lstm_out, tags, mask=None, reduction='mean')

    def predict(self, sentences):
        lstm_out = self.forward(sentences)
        return self.crf.decode(lstm_out)

sentences = []
tags = []
with open("/content/train","r") as f:
    sentence,tag1 = [],[]
    for line in f.readlines():
        if(len(line.strip())==0):
            sentences.append(sentence)
            tags.append(tag1)
            tag1 = []
            sentence = []
        else:
            sent,tag = line.strip().split("	")
            sentence.append(sent)
            tag1.append(tag)

max_train = 500
sentences = sentences[:max_train]
tags = tags[:max_train]

word_vocab = set([word for sentence in sentences for word in sentence])
tag_vocab = set([tag for tag_list in tags for tag in tag_list])

word2idx = {word: idx+1 for idx, word in enumerate(word_vocab)}
word2idx['<PAD>'] = 0
tag2idx = {tag: idx for idx, tag in enumerate(tag_vocab)}

X_data = [[word2idx[word] for word in sentence] for sentence in sentences]
y_data = [[tag2idx[tag] for tag in tag_list] for tag_list in tags]

max_len = max(len(sentence) for sentence in X_data)
X_data = [sentence + [0] * (max_len - len(sentence)) for sentence in X_data]
y_data = [tag_list + [tag2idx['O']] * (max_len - len(tag_list)) for tag_list in y_data]

X_tensor = torch.tensor(X_data, dtype=torch.long)
y_tensor = torch.tensor(y_data, dtype=torch.long)

batch_size = 2
train_data = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_dim = 100
hidden_dim = 256
tagset_size = len(tag2idx)
padding_idx = word2idx['<PAD>']

model = BiLSTM_CRF(vocab_size=len(word2idx), embedding_dim=embedding_dim, hidden_dim=hidden_dim, tagset_size=tagset_size, padding_idx=padding_idx)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5

from tqdm import tqdm

for epoch in range(epochs):
    model.train()
    total_loss = 0
    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as batch_bar:
        for batch_sentences, batch_tags in batch_bar:
            batch_sentences = batch_sentences.to(device)
            batch_tags = batch_tags.to(device)

            model.zero_grad()
            loss = model.neg_log_likelihood(batch_sentences, batch_tags)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()
            batch_bar.set_postfix(loss=total_loss / (batch_bar.n + 1))

    print(device)
    print(f"Epoch {epoch+1}/{epochs} - Total Loss: {total_loss:.4f}")

model.eval()

Epoch 1/5:   0%|          | 0/250 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:530.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)
Epoch 1/5: 100%|██████████| 250/250 [00:25<00:00,  9.97batch/s, loss=32.1]


cuda
Epoch 1/5 - Total Loss: 8017.7718


Epoch 2/5: 100%|██████████| 250/250 [00:18<00:00, 13.48batch/s, loss=13.5]


cuda
Epoch 2/5 - Total Loss: 3357.2539


Epoch 3/5: 100%|██████████| 250/250 [00:17<00:00, 14.00batch/s, loss=8.44]


cuda
Epoch 3/5 - Total Loss: 2101.1767


Epoch 4/5: 100%|██████████| 250/250 [00:18<00:00, 13.36batch/s, loss=4.97]


cuda
Epoch 4/5 - Total Loss: 1241.8988


Epoch 5/5: 100%|██████████| 250/250 [00:17<00:00, 13.96batch/s, loss=2.94]


cuda
Epoch 5/5 - Total Loss: 731.9922
Predictions for 'John is from New York': ['O', 'O', 'O', 'O', 'O']


In [3]:
import torch

def load_data(file_path):
    sentences, tags = [], []
    with open(file_path, "r") as f:
        sentence, tag1 = [], []
        for line in f.readlines():
            if len(line.strip()) == 0:
                sentences.append(sentence)
                tags.append(tag1)
                sentence, tag1 = [], []
            else:
                try:
                    sent, tag = line.strip().split("\t")
                except ValueError:
                    sent = line.strip()
                    tag = 'O'
                sentence.append(sent)
                tag1.append(tag)
    return sentences, tags

def predict_and_save(model, file_path, output_file, word2idx, tag2idx, max_len_train, device):
    sentences, actual_tags = load_data(file_path)
    X_data = [[word2idx.get(word, word2idx['<PAD>']) for word in sentence] for sentence in sentences]
    max_len_current = max(len(sentence) for sentence in X_data)
    max_len = max(max_len_train, max_len_current)

    X_data = [sentence + [0] * (max_len - len(sentence)) for sentence in X_data]
    X_tensor = torch.tensor(X_data, dtype=torch.long).to(device)

    model.eval()
    predictions = model.predict(X_tensor)
    idx2tag = {idx: tag for tag, idx in tag2idx.items()}
    predicted_tags = [[idx2tag[tag] for tag in pred] for pred in predictions]

    with open(output_file, "w") as f:
        for i in range(len(sentences)):
            pred_tags_i = predicted_tags[i][:len(sentences[i])]
            actual_tags_i = actual_tags[i][:len(sentences[i])]
            for word, pred_tag, actual_tag in zip(sentences[i], pred_tags_i, actual_tags_i):
                f.write(f"{word}\t{pred_tag}\n")
            f.write("\n")

predict_and_save(model, "/content/test", "/content/test_1_output", word2idx, tag2idx, max_len, device)
predict_and_save(model, "/content/dev", "/content/dev_1_output", word2idx, tag2idx, max_len, device)

print("Predictions saved to test.output and dev.output")

Predictions saved to test.output and dev.output


In [4]:
from sklearn.metrics import classification_report

def load_tags(pred_file, actual_file):
    """Loads predicted and actual tags from separate files."""
    predicted, actual = [], []

    with open(pred_file, "r") as f:
        pred = []
        for line in f:
            if line.strip():
                _, pred_tag = line.strip().split("\t")[:2]
                pred.append(pred_tag)
            else:
                predicted.append(pred)
                pred = []

    with open(actual_file, "r") as f:
        act = []
        for line in f:
            if line.strip():
                _, actual_tag = line.strip().split("\t")[:2]
                act.append(actual_tag)
            else:
                actual.append(act)
                act = []

    return actual, predicted

def evaluate(pred_file, actual_file):
    actual, predicted = load_tags(pred_file, actual_file)

    actual_flat = [tag for sentence in actual for tag in sentence]
    predicted_flat = [tag for sentence in predicted for tag in sentence]

    print(f"Evaluation for {pred_file}:")
    print(classification_report(actual_flat, predicted_flat, digits=4))

evaluate("/content/test_1_output", "test.answers")
evaluate("/content/dev_1_output", "dev.answers")

Evaluation for /content/dev_1_output:
              precision    recall  f1-score   support

       B-DNA     0.1538    0.3333    0.2105         6
       B-RNA     0.0000    0.0000    0.0000        15
 B-cell_line     0.1429    0.5000    0.2222         2
 B-cell_type     0.7200    0.3214    0.4444        56
   B-protein     0.4253    0.2984    0.3507       124
       I-DNA     0.3889    0.7778    0.5185         9
       I-RNA     0.5000    0.0769    0.1333        26
 I-cell_line     0.1364    0.5000    0.2143         6
 I-cell_type     0.7955    0.4070    0.5385        86
   I-protein     0.7119    0.2781    0.4000       151
           O     0.8908    0.9677    0.9277      2319

    accuracy                         0.8539      2800
   macro avg     0.4423    0.4055    0.3600      2800
weighted avg     0.8405    0.8539    0.8348      2800



In [1]:
# Character level CNN implementation

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchcrf import CRF
from torch.utils.data import DataLoader, TensorDataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class CharCNNEmbedding(nn.Module):
    def __init__(self, char_vocab_size, char_embedding_dim, char_kernel_size, char_filters):
        super(CharCNNEmbedding, self).__init__()
        self.char_embedding = nn.Embedding(char_vocab_size, char_embedding_dim, padding_idx=0)
        self.conv = nn.Conv1d(char_embedding_dim, char_filters, kernel_size=char_kernel_size)
        self.maxpool = nn.AdaptiveMaxPool1d(1)

    def forward(self, x):
        batch_size, seq_len, word_len = x.size()
        x = self.char_embedding(x).view(batch_size * seq_len, word_len, -1).permute(0, 2, 1)
        x = F.relu(self.conv(x))
        x = self.maxpool(x).squeeze(2)
        return x.view(batch_size, seq_len, -1)

class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size, padding_idx, char_vocab_size, char_embedding_dim, char_hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.char_embedding = CharCNNEmbedding(char_vocab_size, char_embedding_dim, 3, char_hidden_dim)
        self.lstm = nn.LSTM(embedding_dim + char_hidden_dim, hidden_dim // 2, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.crf = CRF(tagset_size, batch_first=True)

    def forward(self, sentences, char_inputs):
        word_embeds = self.word_embedding(sentences)
        char_embeds = self.char_embedding(char_inputs)
        combined_embeds = torch.cat([word_embeds, char_embeds], dim=-1)
        lstm_out, _ = self.lstm(combined_embeds)
        tag_scores = self.hidden2tag(lstm_out)
        return tag_scores

    def neg_log_likelihood(self, sentences, tags, char_inputs):
        emissions = self.forward(sentences, char_inputs)
        mask = sentences != 0
        return -self.crf(emissions, tags, mask=mask, reduction='mean')

    def predict(self, sentences, char_inputs):
        emissions = self.forward(sentences, char_inputs)
        return self.crf.decode(emissions)


sentences = []
tags = []
with open("/content/train","r") as f:
    sentence,tag1 = [],[]
    for line in f.readlines():
        if(len(line.strip())==0):
            sentences.append(sentence)
            tags.append(tag1)
            tag1 = []
            sentence = []
        else:
            sent,tag = line.strip().split("	")
            sentence.append(sent)
            tag1.append(tag)

max_train = 500
sentences = sentences[:max_train]
tags = tags[:max_train]

word_vocab = {word: idx + 1 for idx, word in enumerate(set(word for sent in sentences for word in sent))}
word_vocab['<PAD>'] = 0

tag_vocab = {tag: idx for idx, tag in enumerate(set(tag for tag_seq in tags for tag in tag_seq))}

char_vocab = {char: idx + 1 for idx, char in enumerate(set(char for word in word_vocab for char in word))}
char_vocab['<PAD>'] = 0

X_data = [[word_vocab[word] for word in sentence] for sentence in sentences]
y_data = [[tag_vocab[tag] for tag in tag_seq] for tag_seq in tags]

char_data = [[[char_vocab[char] for char in word] for word in sentence] for sentence in sentences]

max_word_len = max(len(word) for sentence in char_data for word in sentence)
max_sentence_len = max(len(sentence) for sentence in X_data)

X_data = [sentence + [0] * (max_sentence_len - len(sentence)) for sentence in X_data]
y_data = [tag_seq + [0] * (max_sentence_len - len(tag_seq)) for tag_seq in y_data]
char_data = [[word + [0] * (max_word_len - len(word)) for word in sentence] for sentence in char_data]
char_data = [sentence + [[0] * max_word_len] * (max_sentence_len - len(sentence)) for sentence in char_data]

X_tensor = torch.tensor(X_data, dtype=torch.long)
y_tensor = torch.tensor(y_data, dtype=torch.long)
char_tensor = torch.tensor(char_data, dtype=torch.long)

X_tensor = X_tensor.to(device)
y_tensor = y_tensor.to(device)
char_tensor = char_tensor.to(device)

vocab_size = len(word_vocab)
char_vocab_size = len(char_vocab)
embedding_dim = 100
char_embedding_dim = 30
char_hidden_dim = 50
hidden_dim = 256
tagset_size = len(tag_vocab)
padding_idx = word_vocab['<PAD>']

model = BiLSTM_CRF(vocab_size, embedding_dim, hidden_dim, tagset_size, padding_idx, char_vocab_size, char_embedding_dim, char_hidden_dim)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
model.train()
optimizer.zero_grad()
loss = model.neg_log_likelihood(X_tensor, y_tensor, char_tensor)
loss.backward()
optimizer.step()

model.eval()

predictions = model.predict(test_sentence_tensor, test_char_tensor)
predicted_tags = [list(tag_vocab.keys())[tag] for tag in predictions[0]]
print(predicted_tags)

['B-protein', 'O', 'B-DNA', 'B-cell_type', 'O']


/usr/local/lib/python3.11/dist-packages/torchcrf/__init__.py:305: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:530.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


In [3]:
import torch

def load_data(file_path):
    sentences, tags = [], []
    with open(file_path, "r") as f:
        sentence, tag1 = [], []
        for line in f.readlines():
            if len(line.strip()) == 0:
                sentences.append(sentence)
                tags.append(tag1)
                sentence, tag1 = [], []
            else:
                try:
                    sent, tag = line.strip().split("\t")
                except ValueError:
                    sent = line.strip()
                    tag = 'O'
                sentence.append(sent)
                tag1.append(tag)
    return sentences, tags

def predict_and_save(model, file_path, output_file, word2idx, tag2idx, char2idx, max_word_len, max_len_train, device, batch_size=8):
    sentences, actual_tags = load_data(file_path)

    X_data = [[word2idx.get(word, 0) for word in sentence] for sentence in sentences]
    max_len_current = max(len(sentence) for sentence in X_data)
    max_len = max(max_len_train, max_len_current)

    char_data = [[[char2idx.get(char, 0) for char in word] for word in sentence] for sentence in sentences]
    char_data = [[word + [0] * (max_word_len - len(word)) for word in sentence] for sentence in char_data]
    char_data = [sentence + [[0] * max_word_len] * (max_len - len(sentence)) for sentence in char_data]

    X_data = [sentence + [0] * (max_len - len(sentence)) for sentence in X_data]

    model.eval()
    all_predictions = []

    with torch.no_grad():
        for i in range(0, len(X_data), batch_size):
            batch_X = X_data[i:min(i+batch_size, len(X_data))]
            batch_char = char_data[i:min(i+batch_size, len(char_data))]

            X_tensor = torch.tensor(batch_X, dtype=torch.long).to(device)
            char_tensor = torch.tensor(batch_char, dtype=torch.long).to(device)

            batch_predictions = model.predict(X_tensor, char_tensor)
            all_predictions.extend(batch_predictions)

            del X_tensor, char_tensor
            torch.cuda.empty_cache()

    idx2tag = {idx: tag for tag, idx in tag2idx.items()}
    predicted_tags = [[idx2tag[tag] for tag in pred] for pred in all_predictions]

    with open(output_file, "w") as f:
        for i in range(len(sentences)):
            pred_tags_i = predicted_tags[i][:len(sentences[i])]
            actual_tags_i = actual_tags[i][:len(sentences[i])]
            for word, pred_tag, actual_tag in zip(sentences[i], pred_tags_i, actual_tags_i):
                f.write(f"{word}\t{pred_tag}\n")
            f.write("\n")

batch_size = 16

predict_and_save(model, "/content/test", "/content/test_2_output", word_vocab, tag_vocab, char_vocab, max_word_len, max_sentence_len, device, batch_size)
predict_and_save(model, "/content/dev", "/content/dev_2_output", word_vocab, tag_vocab, char_vocab, max_word_len, max_sentence_len, device, batch_size)

print("Predictions saved to test_2_output and dev_2_output")

Predictions saved to test_2_output and dev_2_output


In [5]:
from sklearn.metrics import classification_report

def load_tags(pred_file, actual_file):
    predicted, actual = [], []

    with open(pred_file, "r") as f:
        pred = []
        for line in f:
            if line.strip():
                _, pred_tag = line.strip().split("\t")[:2]
                pred.append(pred_tag)
            else:
                predicted.append(pred)
                pred = []

    with open(actual_file, "r") as f:
        act = []
        for line in f:
            if line.strip():
                _, actual_tag = line.strip().split("\t")[:2]
                act.append(actual_tag)
            else:
                actual.append(act)
                act = []

    return actual, predicted

def evaluate(pred_file, actual_file):
    actual, predicted = load_tags(pred_file, actual_file)
    predicted = predicted[:len(actual)]
    print(len(actual))
    print(len(predicted))
    actual_flat = [tag for sentence in actual for tag in sentence]
    predicted_flat = [tag for sentence in predicted for tag in sentence]

    print(f"Evaluation for {pred_file}:")
    print(classification_report(actual_flat, predicted_flat, digits=4))

evaluate("/content/test_2_output", "test.answers")
evaluate("/content/dev_2_output", "dev.answers")

3855
3855
Evaluation for /content/test_2_output:
              precision    recall  f1-score   support

       B-DNA     0.0047    0.0502    0.0085      1056
       B-RNA     0.0000    0.0000    0.0000       118
 B-cell_line     0.0026    0.0020    0.0023       497
 B-cell_type     0.0196    0.1615    0.0350      1919
   B-protein     0.0871    0.2041    0.1220      5067
       I-DNA     0.0146    0.0056    0.0081      1789
       I-RNA     0.0019    0.0053    0.0028       187
 I-cell_line     0.0050    0.0020    0.0029       983
 I-cell_type     0.0097    0.0010    0.0018      2987
   I-protein     0.0575    0.0172    0.0265      4774
           O     0.8047    0.5635    0.6628     81615

    accuracy                         0.4702    100992
   macro avg     0.0916    0.0920    0.0793    100992
weighted avg     0.6584    0.4702    0.5440    100992

105
105
Evaluation for /content/dev_2_output:
              precision    recall  f1-score   support

       B-DNA     0.0000    0.0000    

In [15]:
actual, predicted = load_tags("/content/test_2_output", "test.answers")

In [13]:
# BiLSTM CRF and CNN Implementation

import torch
import torch.nn as nn
import torch.optim as optim
from torchcrf import CRF
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

class BiLSTM_CRF(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size, padding_idx):
        super(BiLSTM_CRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, num_layers=1, bidirectional=True, batch_first=True)
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.crf = CRF(tagset_size, batch_first=True)

    def forward(self, sentences):
        word_embeds = self.embedding(sentences)
        lstm_out, _ = self.lstm(word_embeds)
        lstm_out = self.hidden2tag(lstm_out)
        return lstm_out

    def hamming_loss(self, y_pred, y_true):
        return (y_pred != y_true).float().mean()

    def neg_log_likelihood(self, sentences, tags, loss_type="softmax_margin"):
        lstm_out = self.forward(sentences)
        crf_loss = -self.crf(lstm_out, tags, mask=None, reduction='mean')
        print(f"CRF Loss: {crf_loss.item()}")
        predicted_tags = self.crf.decode(lstm_out)
        predicted_tags = torch.tensor(predicted_tags, dtype=torch.long).to(tags.device)
        hamming_cost = self.hamming_loss(predicted_tags, tags)

        if loss_type == "max_margin":
            margin_loss = torch.max(crf_loss - hamming_cost, torch.tensor(0.0).to(tags.device))
            return margin_loss.mean()

        elif loss_type == "softmax_margin":
            log_sum_exp = torch.logsumexp(lstm_out, dim=2)
            softmax_margin_loss = torch.mean(log_sum_exp - crf_loss + hamming_cost)
            return softmax_margin_loss

        elif loss_type == "ramp_loss":
            ramp_loss = torch.max(crf_loss, torch.tensor(0.0).to(tags.device)) + hamming_cost
            return ramp_loss.mean()

        elif loss_type == "soft_ramp_loss":
            soft_ramp_loss = torch.mean(torch.logsumexp(lstm_out, dim=2) + hamming_cost)
            return soft_ramp_loss

        else:
            return crf_loss


    def predict(self, sentences):
        lstm_out = self.forward(sentences)
        return self.crf.decode(lstm_out)



sentences = []
tags = []
with open("/content/train","r") as f:
    sentence,tag1 = [],[]
    for line in f.readlines():
        if(len(line.strip())==0):
            sentences.append(sentence)
            tags.append(tag1)
            tag1 = []
            sentence = []
        else:
            sent,tag = line.strip().split("	")
            sentence.append(sent)
            tag1.append(tag)

max_train = 500
sentences = sentences[:max_train]
tags = tags[:max_train]

word_vocab = set([word for sentence in sentences for word in sentence])
tag_vocab = set([tag for tag_list in tags for tag in tag_list])

word2idx = {word: idx+1 for idx, word in enumerate(word_vocab)}
word2idx['<PAD>'] = 0
tag2idx = {tag: idx for idx, tag in enumerate(tag_vocab)}

X_data = [[word2idx[word] for word in sentence] for sentence in sentences]
y_data = [[tag2idx[tag] for tag in tag_list] for tag_list in tags]

max_len = max(len(sentence) for sentence in X_data)
X_data = [sentence + [0] * (max_len - len(sentence)) for sentence in X_data]
y_data = [tag_list + [tag2idx['O']] * (max_len - len(tag_list)) for tag_list in y_data]

X_tensor = torch.tensor(X_data, dtype=torch.long)
y_tensor = torch.tensor(y_data, dtype=torch.long)

batch_size = 16
train_data = TensorDataset(X_tensor, y_tensor)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

embedding_dim = 100
hidden_dim = 256
tagset_size = len(tag2idx)
padding_idx = word2idx['<PAD>']

model = BiLSTM_CRF(vocab_size=len(word2idx), embedding_dim=embedding_dim, hidden_dim=hidden_dim, tagset_size=tagset_size, padding_idx=padding_idx)
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 20

from tqdm import tqdm

for epoch in range(epochs):
    model.train()
    total_loss = 0

    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as batch_bar:
        for batch_sentences, batch_tags in batch_bar:
            batch_sentences = batch_sentences.to(device)
            batch_tags = batch_tags.to(device)

            model.zero_grad()

            loss = model.neg_log_likelihood(batch_sentences, batch_tags,loss_type="")
            total_loss += loss.item()

            loss.backward()
            optimizer.step()
            batch_bar.set_postfix(loss=total_loss / (batch_bar.n + 1))

    print(device)
    print(f"Epoch {epoch+1}/{epochs} - Total Loss: {total_loss:.4f}")

model.eval()

print(f"Predictions for '{' '.join(test_sentence)}': {predicted_tags}")

Epoch 1/20:   0%|          | 0/32 [00:00<?, ?batch/s]

CRF Loss: 0.0499725341796875


Epoch 1/20:   6%|▋         | 2/32 [00:00<00:07,  3.84batch/s, loss=0.0702]

CRF Loss: 0.09042739868164062
CRF Loss: 0.06840133666992188


Epoch 1/20:  12%|█▎        | 4/32 [00:00<00:05,  5.52batch/s, loss=0.0675]

...

CRF Loss: 0.002528965473175049
CRF Loss: 0.002125948667526245


Epoch 20/20:  91%|█████████ | 29/32 [00:03<00:00,  9.49batch/s, loss=0.00261]

CRF Loss: 0.0028299614787101746
CRF Loss: 0.00291287899017334
CRF Loss: 0.0023041069507598877


Epoch 20/20: 100%|██████████| 32/32 [00:03<00:00,  9.74batch/s, loss=0.00258]

CRF Loss: 0.0018901824951171875
cuda
Epoch 20/20 - Total Loss: 0.0826
Predictions for 'John is from New York': ['O', 'O', 'O', 'O', 'O']


In [14]:
from sklearn.metrics import classification_report
import torch

def load_data(file_path):
    sentences, tags = [], []
    with open(file_path, "r") as f:
        sentence, tag1 = [], []
        for line in f.readlines():
            if len(line.strip()) == 0:
                if sentence:
                    sentences.append(sentence)
                    tags.append(tag1)
                sentence, tag1 = [], []
            else:
                parts = line.strip().split("\t")
                if len(parts) == 2:
                    sent, tag = parts
                else:
                    sent = parts[0]
                    tag = 'O'

                sentence.append(sent)
                tag1.append(tag)
    return sentences, tags

def save_predictions(model, dataset_path, output_file, word2idx, tag2idx, device, batch_size=8):
    sentences, tags = load_data(dataset_path)
    X_data = [[word2idx.get(word, word2idx['<PAD>']) for word in sentence] for sentence in sentences]
    max_len_current = max(len(sentence) for sentence in X_data)
    X_data = [sentence + [0] * (max_len_current - len(sentence)) for sentence in X_data]

    model.eval()
    all_predictions = []

    with torch.no_grad():
        for i in range(0, len(X_data), batch_size):
            batch_X = X_data[i:min(i+batch_size, len(X_data))]
            X_tensor = torch.tensor(batch_X, dtype=torch.long).to(device)
            batch_predictions = model.predict(X_tensor)
            all_predictions.extend(batch_predictions)
            del X_tensor
            torch.cuda.empty_cache()
    idx2tag = {idx: tag for tag, idx in tag2idx.items()}
    predicted_tags = [[idx2tag[tag] for tag in pred[:len(sentences[i])]] for i, pred in enumerate(all_predictions)]

    with open(output_file, "w") as f_out:
        for sent, pred_tags in zip(sentences, predicted_tags):
            for word, tag in zip(sent, pred_tags):
                f_out.write(f"{word}\t{tag}\n")
            f_out.write("\n")

    return sentences, tags, predicted_tags

def evaluate_model(true_labels, pred_labels):
    """ Compute classification metrics: Precision, Recall, F1-score """
    true_labels_flat = [tag for sent in true_labels for tag in sent]
    pred_labels_flat = [tag for sent in pred_labels for tag in sent]

    print("\nClassification Report:")
    print(classification_report(true_labels_flat, pred_labels_flat, zero_division=0))

batch_size = 16

_, true_tags_dev, pred_tags_dev = save_predictions(
    model,
    "/content/dev",
    "dev.output_3",
    word2idx,
    tag2idx,
    device,
    batch_size
)

print("Processing test set...")
_, true_tags_test, pred_tags_test = save_predictions(
    model,
    "/content/test",
    "test.output_3",
    word2idx,
    tag2idx,
    device,
    batch_size
)

print("Evaluating dev set...")
evaluate_model(true_tags_dev, pred_tags_dev)

print("Evaluating test set...")
evaluate_model(true_tags_test, pred_tags_test)

Processing dev set...
Processing test set...
Evaluating dev set...

Classification Report:
              precision    recall  f1-score   support

       B-DNA       0.00      0.00      0.00         0
       B-RNA       0.00      0.00      0.00         0
 B-cell_line       0.00      0.00      0.00         0
 B-cell_type       0.00      0.00      0.00         0
   B-protein       0.00      0.00      0.00         0
       I-DNA       0.00      0.00      0.00         0
       I-RNA       0.00      0.00      0.00         0
 I-cell_line       0.00      0.00      0.00         0
 I-cell_type       0.00      0.00      0.00         0
   I-protein       0.00      0.00      0.00         0
           O       1.00      0.89      0.94      2800

    accuracy                           0.89      2800
   macro avg       0.09      0.08      0.09      2800
weighted avg       1.00      0.89      0.94      2800

Evaluating test set...

Classification Report:
              precision    recall  f1-score   sup